In [3]:
import csv
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
from shapely.geometry import Point, Polygon

In [108]:
# this is to read all the needed datasets
workbook =pd.read_excel('../data/landing/Book2.xlsx')
pop_column = workbook['SA2_code_population ']
wealth_column= workbook['SA2_code_wealth']
suburb_column = workbook["suburb"]
bus_data = gpd.read_file('../data/landing/public_transport/bus/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_METRO_BUS_STOP.shp')
tram_data= gpd.read_file('../data/landing/public_transport/tram/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_METRO_TRAM_STOP.shp')
train_data = gpd.read_file('../data/landing/public_transport/metro_train/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_METRO_TRAIN_STATION.shp')
geopackage_path = "../data/landing/Geopackage/SA2 ERP GeoPackage 2022.gpkg"
SA2_code_data = gpd.read_file(geopackage_path)


In [ ]:
# this is to get specific geometry of the selected SA2 areas from book2.xlsx
SA2_list = []
for i in pop_column:
    if len(str(i))>9: 
        code_list = i.split(",")
        for code  in code_list:
            SA2_list.append(int(code))
    else:
        if len(str(i))==9:
            SA2_list.append(i)
SA2_code_data_selected_row = SA2_code_data[SA2_code_data["State_name_2021"]=="Victoria"]
SA2_code_data_selected_row=SA2_code_data_selected_row.reset_index(drop=True)
SA2_code_data_selected_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"].isin(SA2_list)]

In [ ]:
# this is to get the number of bus stops in each selected sa2 areas 
value_list = []
for i in pop_column:
    # this means that there is more than one SA2 code present in a box 
    # for one sa2 code 

    if len(str(i))==9:
        counts= 0 
        geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(i)]
        geometry = geo_row["geometry"]
        for count  in range(len(bus_data)):
            p = bus_data["geometry"][count]
            is_inside = geometry.contains(p).any()
            if  is_inside: 
                counts+=1
        value_list.append(counts)
    
    if len(str(i))>9:
        number_of_sa2 = 0
        code_list = i.split(",") 
        counts=0
        for code  in code_list:
            geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(code)]
            geometry = geo_row["geometry"]
            number_of_sa2 +=1
            for count  in range(len(bus_data)):
                p = bus_data["geometry"][count]
                is_inside = geometry.contains(p).any()
                if  is_inside: 
                    counts+=1
        counts = counts/number_of_sa2
        value_list.append(counts)
    

In [134]:
# this is to get the number of train stops in each selected sa2 areas 
value_list_train = []
for i in pop_column:
    # this means that there is more than one SA2 code present in a box 
    # for one sa2 code 
    if len(str(i))==9:
        counts= 0 
        geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(i)]
        geometry = geo_row["geometry"]
        for count  in range(len(train_data)):
            p = train_data["geometry"][count]
            is_inside = geometry.contains(p).any()
            if  is_inside: 
                counts+=1
        value_list_train.append(counts)
    
    if len(str(i))>9:
        number_of_sa2 = 0
        code_list = i.split(",") 
        counts=0
        for code  in code_list:
            geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(code)]
            geometry = geo_row["geometry"]
            number_of_sa2 +=1
 
            for count  in range(len(train_data)):
                p = train_data["geometry"][count]
                
                is_inside = geometry.contains(p).any()
                if  is_inside: 
                    counts+=1
        counts = counts/number_of_sa2
        value_list_train .append(counts)


In [135]:
# this is to get the number of tram stops in each selected sa2 areas 
value_list_tram = []
for i in pop_column:
    # this means that there is more than one SA2 code present in a box 
    # for one sa2 code 
    if len(str(i))==9:
        counts= 0 
        geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(i)]
        geometry = geo_row["geometry"]
        for count  in range(len(tram_data)):
            p = tram_data["geometry"][count]
            is_inside = geometry.contains(p).any()
            if  is_inside: 
                counts+=1
        value_list_tram.append(counts)
    
    if len(str(i))>9:
        number_of_sa2 = 0
        code_list = i.split(",") 
        counts=0
        for code  in code_list:
            geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(code)]
            geometry = geo_row["geometry"]
            number_of_sa2 +=1
            for count  in range(len(tram_data)):
                p = tram_data["geometry"][count]
                is_inside = geometry.contains(p).any()
                if  is_inside: 
                    counts+=1
        counts = counts/number_of_sa2
        value_list_tram .append(counts)
                


In [138]:
# hospital data 
hospital_data = pd.read_csv("../data/raw/hospital_address.csv")
# shopping centre data 
shopping_centre_data = pd.read_csv("../data/raw/shopping_centers.csv")



In [143]:
# this is to get the number of hospital in each selected sa2 areas 
hospital_value_list =[]
for i in pop_column:
    
    if len(str(i))==9:
        geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(i)]
        geometry = geo_row["geometry"]
        number_hospital= 0 
        for count in range(len(hospital_data)):
            long=hospital_data["longtitude"][count]
            lat=hospital_data["latitude"][count]
            point =Point(long,lat)
            is_inside = geometry.contains(point).any()
            if is_inside:
                number_hospital+=1
        hospital_value_list.append(number_hospital)
    
    
    if len(str(i))>9:
        number_of_sa2 = 0
        code_list = i.split(",") 
        number_hospital=0
        for code  in code_list:
            geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(code)]
            geometry = geo_row["geometry"]
            number_of_sa2 +=1
            for count in range(len(hospital_data)):
                long=hospital_data["longtitude"][count]
                lat=hospital_data["latitude"][count]
                point =Point(long,lat)
                is_inside = geometry.contains(point).any()
                if is_inside:
                    number_hospital+=1
        hospital_value_list.append(number_hospital/number_of_sa2)

            
        
    

In [151]:
# this is to get the number of shopping centre in each selected sa2 areas 
shopping_centre_value_list =[]
for i in pop_column:
    
    if len(str(i))==9:
        geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(i)]
        geometry = geo_row["geometry"]
        number_shopping_center = 0 
        for count in range(len(shopping_centre_data)):
            long = shopping_centre_data["GPS Information"][count].split(",")[1]
            lat = shopping_centre_data["GPS Information"][count].split(",")[0]
            point = Point(long,lat)
            is_inside = geometry.contains(point).any()
            if is_inside:
                number_shopping_center+=1
        shopping_centre_value_list.append(number_shopping_center)
    
    if len(str(i))>9:
        number_of_sa2 = 0
        code_list = i.split(",") 
        number_shopping_center=0
        for code  in code_list:
            geo_row = SA2_code_data_selected_row[SA2_code_data_selected_row["SA2_code_2021"]==int(code)]
            geometry = geo_row["geometry"]
            number_of_sa2 +=1
            for count in range(len(shopping_centre_data)):
                long = shopping_centre_data["GPS Information"][count].split(",")[1]
                lat = shopping_centre_data["GPS Information"][count].split(",")[0]
                point = Point(long,lat)
                is_inside = geometry.contains(point).any()
                if is_inside:
                    number_shopping_center+=1
        shopping_centre_value_list.append(number_shopping_center/number_of_sa2)
                    

            
        
    

In [154]:
# this is to gather all the collected information and form =ed in to a csv for analysing later. 
column_name = ["suburb","SA2_code","number_bus","number_train","number_train"]
for i in range (6):
    hospital_value_list.append("")
    shopping_centre_value_list.append("")
    
data = {"suburb":suburb_column,"SA2_code":pop_column,"number_bus":value_list,"number_train":value_list_train,
        "number_tram":value_list_tram,"number_hospital":hospital_value_list,"number_shopping_center":shopping_centre_value_list}
df = pd.DataFrame(data)
df.to_csv("../data/raw/transport_in_suburb.csv")